# La idea de este notebook es depurar ouliers, malos datos de transacciones y enriquecer el texto con un PoS tag. Para esto es necesario un trabajo idiomatico en estos campos.

In [1]:
import gensim as gs
import nltk
import pandas as pd
import numpy as np
import spacy as spc
import re

D:\Anaconda\envs\dataton2\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
%matplotlib inline

In [3]:
!dir

 Volume in drive D is DATA
 Volume Serial Number is DA3C-2954

 Directory of D:\Local_Dataton

10/28/2018  08:34 AM    <DIR>          .
10/28/2018  08:34 AM    <DIR>          ..
10/28/2018  12:38 AM    <DIR>          .ipynb_checkpoints
10/28/2018  12:00 AM         4,530,564 DataExplorationAndTransformation.ipynb
10/27/2018  11:45 PM            47,040 DataTypes_Quality.ipynb
10/27/2018  01:19 PM       965,521,031 dt_trxpse_V2.pkl.gzip
10/28/2018  08:34 AM            44,954 TrxDepuration_RefPoS.ipynb
10/28/2018  12:18 AM                72 Untitled.ipynb
               5 File(s)    970,143,661 bytes
               3 Dir(s)  1,766,791,249,920 bytes free


# Cargo el corpus idiomatico y checkeo que funcione

In [4]:
nlp_multi = spc.load('es_core_news_sm')
doc_multi = nlp_multi(u"pago por payu falabella ")
print([(ent.text, ent.label) for ent in doc_multi.ents])

[]


In [5]:
nlp = spc.load('es_core_news_sm')
doc = nlp(u"pago por payu falabella ")
print([(w.text, w.pos_) for w in doc])
print([(ent.text, ent.label) for ent in doc.ents])

[('pago', 'NOUN'), ('por', 'ADP'), ('payu', 'ADJ'), ('falabella', 'NOUN')]
[]


In [6]:
data = pd.read_pickle('dt_trxpse_V2.pkl.gzip')

In [7]:
data.head()

,id_trn_ach,id_cliente,valor_trx,ref1,ref2,ref3,sector,subsector,descripcion,fecha_usable
0,230435642,3,2122392.51,CC,NaN,NaN,NaN,NaN,NaN,2016-12-07 11:34:51
1,222356110,10,148438.37,Referencia: Contrato: Valor:,CC,NaN,NaN,NaN,NaN,2016-10-16 00:34:24
2,309137749,10,94025.19,CC,NaN,NaN,NaN,NaN,NaN,2018-01-20 19:50:42
3,324614737,10,94430.07,CC,NaN,NaN,NaN,NaN,NaN,2018-03-26 19:21:46
4,235344690,18,670645.57,MEDICINA PREPAGADA COLSANITAS,CE,NaN,NaN,NaN,NaN,2017-01-06 20:13:17


# A continuacion se trabaja sobre el campo Ref1

Para poder aprovechar los beneficios del NLP, es necesario depurar los valores que no son utiles.

In [8]:
import unidecode

In [9]:
def decode_word(x):
    try:
        return unidecode.unidecode(x)
    except:
        return x

In [10]:
data['ref1_notilde'] = data['ref1'].apply(lambda x: decode_word(x))
data['ref2_notilde'] = data['ref2'].apply(lambda x: decode_word(x))
data['ref3_notilde'] = data['ref2'].apply(lambda x: decode_word(x))

# Luego de deshacernos de la tilde, podemos deshacernos de los caracteres especiales

In [11]:
def no_especial_chars_word(x):
    try:
        return ' '.join(re.sub('[^a-zA-Z0-9 \n\.]', ' ', x).lower().split())
    except:
        return x

In [12]:
data['ref1_notilde_noespecial'] = data['ref1_notilde'].apply(lambda x: no_especial_chars_word(x))
data['ref2_notilde_noespecial'] = data['ref2_notilde'].apply(lambda x: no_especial_chars_word(x))
data['ref3_notilde_noespecial'] = data['ref3_notilde'].apply(lambda x: no_especial_chars_word(x))

# Como la idea es aplicar de manera sistematica el analisis, en este caso se va a proceder a reemplazar los nan's por "vacio" como string.

In [13]:
data['ref1_notilde_noespecial_vacio'] = data['ref1_notilde_noespecial'].fillna('vacio')
data['ref2_notilde_noespecial_vacio'] = data['ref2_notilde_noespecial'].fillna('vacio')
data['ref3_notilde_noespecial_vacio'] = data['ref3_notilde_noespecial'].fillna('vacio')

# SI escalamos esta funcion en todos los datos vamos a tardar muchisimo tiempo, por lo que vamos a aplicar una reduccion de datos para intentar mejorar el tiempo de la funcion.

In [14]:
data['largo_ref1'] = data['ref1_notilde_noespecial_vacio'].apply(lambda x: len(x.split()))
data['largo_ref2'] = data['ref2_notilde_noespecial_vacio'].apply(lambda x: len(x.split()))
data['largo_ref3'] = data['ref3_notilde_noespecial_vacio'].apply(lambda x: len(x.split()))

In [15]:
doc = nlp(data['ref1_notilde_noespecial_vacio'][33])
print([(w.text, w.pos_) for w in doc])

[('ref', 'NOUN'), ('pago', 'ADJ'), ('express', 'VERB'), ('no', 'ADV')]


In [16]:
nlp = spc.load('es_core_news_sm',parser=False, entity=False)
def list_nlp_filter(x):
    nlp_list = []
    doc = nlp(x)
    
    return [w.text for w in doc if w.pos_ in ["NOUN","PRON","ADJ","VERB"]]

In [17]:
lista_nlp = list_nlp_filter(data['ref1_notilde_noespecial_vacio'][17])

In [18]:
lista_nlp

['pago', 'irecaudo', 'soluciones', 'moviles']

In [29]:
short_index = []
for i , row in enumerate(data['ref1_notilde_noespecial_vacio'].unique()):
    if len(row.split(' ')) > 1 :
        short_index.append(i)

# Se puede depurar aun mas!

# El approach tomado es similar a un map reduce map, la idea es solo computar a valores unicos y luego usarlo como consulta, pasamos de computar 7.5M a computar solo 430K

In [30]:
len(short_index)

432417

In [31]:
curated_transform_values = data['ref1_notilde_noespecial_vacio'].unique()[short_index]

In [22]:
transform_dict_nlp = {}
for text in curated_transform_values:
    transform_dict_nlp[text] = list_nlp_filter(text)

In [23]:
transform_dict_nlp

{'referencia contrato valor': ['referencia', 'contrato', 'valor'],
 'medicina prepagada colsanitas': ['medicina', 'prepagada', 'colsanitas'],
 'recaudo colsanitas': ['recaudo', 'colsanitas'],
 'pago de la planilla cesantias': ['pago', 'planilla', 'cesantias'],
 'pago de la factura conjunto residencial pietra santa ph': ['pago',
  'factura',
  'conjunto',
  'residencial',
  'pietra',
  'santa'],
 'pago de saldo plan familiar': ['pago', 'saldo', 'plan', 'familiar'],
 'pago por irecaudo de claro soluciones moviles': ['pago',
  'irecaudo',
  'soluciones',
  'moviles'],
 'pago factura asociado': ['pago', 'factura', 'asociado'],
 'presentacion y pago del impuesto predial unificado': ['presentacion',
  'pago',
  'impuesto',
  'predial',
  'unificado'],
 'edif mirabell ph bog': ['edif', 'mirabell', 'bog'],
 'liquidacion iuva no': ['liquidacion', 'iuva'],
 'pago de saldo': ['pago', 'saldo'],
 'referencia de pago express no': ['referencia', 'pago', 'express'],
 'referencia pago express no': ['re

In [38]:
short_index_ref2 = []
for i , row in enumerate(data['ref2_notilde_noespecial_vacio'].unique()):
    if len(row.split(' ')) > 1 :
        short_index_ref2.append(i)

In [39]:
len(short_index_ref2)

137118

In [33]:
curated_transform_values_ref2 = data['ref2_notilde_noespecial_vacio'].unique()[short_index_ref2]

In [34]:
transform_dict_nlp_ref2 = {}
for text in curated_transform_values_ref2:
    transform_dict_nlp_ref2[text] = list_nlp_filter(text)

# En el ref2 hay 137118 datos de interes

In [40]:
short_index_ref3 = []
for i , row in enumerate(data['ref3_notilde_noespecial_vacio'].unique()):
    if len(row.split(' ')) > 1 :
        short_index_ref3.append(i)

In [41]:
len(short_index_ref3)

137118

In [37]:
transform_dict_nlp_ref2

{'cedula de ciudadania': ['cedula', 'ciudadania'],
 'aaaxdft null': ['aaaxdft', 'null'],
 'irma fortun de mejia': ['irma', 'fortun', 'mejia'],
 'irma de mejia': ['irma', 'mejia'],
 'aaaslmr null': ['aaaslmr', 'null'],
 'eticket avianca phvk': ['eticket', 'phvk'],
 'eticket avianca nbigju': ['eticket', 'nbigju'],
 'eticket avianca ucuvh': ['eticket', 'ucuvh'],
 'aaabpcn null': ['aaabpcn', 'null'],
 'blb null': ['blb', 'null'],
 'pago de obligacion': ['pago', 'obligacion'],
 'pa u': [],
 'identificador unico de cliente identificador unico de cliente': ['identificador',
  'unico',
  'cliente',
  'identificador',
  'unico',
  'cliente'],
 'eticket avianca tfm': ['eticket', 'tfm'],
 'no tipificado': ['tipificado'],
 'eticket avianca wkow': ['eticket', 'wkow'],
 'bsu null': ['null'],
 'eticket avianca wydhof': ['eticket', 'wydhof'],
 'cc a': ['cc'],
 'eticket avianca oulen': ['eticket', 'oulen'],
 'cedula de extranjeria': ['cedula', 'extranjeria'],
 'raul martin leo': ['raul', 'martin', 'leo

In [42]:
curated_transform_values_ref3 = data['ref3_notilde_noespecial_vacio'].unique()[short_index_ref3]

In [43]:
transform_dict_nlp_ref3 = {}
for text in curated_transform_values_ref3:
    transform_dict_nlp_ref3[text] = list_nlp_filter(text)

In [44]:
transform_dict_nlp_ref3

{'cedula de ciudadania': ['cedula', 'ciudadania'],
 'aaaxdft null': ['aaaxdft', 'null'],
 'irma fortun de mejia': ['irma', 'fortun', 'mejia'],
 'irma de mejia': ['irma', 'mejia'],
 'aaaslmr null': ['aaaslmr', 'null'],
 'eticket avianca phvk': ['eticket', 'phvk'],
 'eticket avianca nbigju': ['eticket', 'nbigju'],
 'eticket avianca ucuvh': ['eticket', 'ucuvh'],
 'aaabpcn null': ['aaabpcn', 'null'],
 'blb null': ['blb', 'null'],
 'pago de obligacion': ['pago', 'obligacion'],
 'pa u': [],
 'identificador unico de cliente identificador unico de cliente': ['identificador',
  'unico',
  'cliente',
  'identificador',
  'unico',
  'cliente'],
 'eticket avianca tfm': ['eticket', 'tfm'],
 'no tipificado': ['tipificado'],
 'eticket avianca wkow': ['eticket', 'wkow'],
 'bsu null': ['null'],
 'eticket avianca wydhof': ['eticket', 'wydhof'],
 'cc a': ['cc'],
 'eticket avianca oulen': ['eticket', 'oulen'],
 'cedula de extranjeria': ['cedula', 'extranjeria'],
 'raul martin leo': ['raul', 'martin', 'leo

In [45]:
def busqueda_nlp(x,dict_nlp):
    try:
        return dict_nlp[x]
    except:
        return x

In [47]:
data['ref1_exp_nlp'] = data['ref1_notilde_noespecial_vacio'].apply(lambda x: busqueda_nlp(x,transform_dict_nlp))
data['ref2_exp_nlp'] = data['ref2_notilde_noespecial_vacio'].apply(lambda x: busqueda_nlp(x,transform_dict_nlp_ref2))
data['ref3_exp_nlp'] = data['ref3_notilde_noespecial_vacio'].apply(lambda x: busqueda_nlp(x,transform_dict_nlp_ref3))

# No se si se llegue a apreciar, pero la reduccion de tiempo es increible, de unas 22 horas en nuestro pc a 15 mins.

In [48]:
data.head()

,id_trn_ach,id_cliente,valor_trx,ref1,ref2,ref3,sector,subsector,descripcion,fecha_usable,...,ref3_notilde_noespecial,ref1_notilde_noespecial_vacio,ref2_notilde_noespecial_vacio,ref3_notilde_noespecial_vacio,largo_ref1,largo_ref2,largo_ref3,ref1_exp_nlp,ref2_exp_nlp,ref3_exp_nlp
0,230435642,3,2122392.51,CC,NaN,NaN,NaN,NaN,NaN,2016-12-07 11:34:51,...,NaN,cc,vacio,vacio,1,1,1,cc,vacio,vacio
1,222356110,10,148438.37,Referencia: Contrato: Valor:,CC,NaN,NaN,NaN,NaN,2016-10-16 00:34:24,...,cc,referencia contrato valor,cc,cc,3,1,1,"[referencia, contrato, valor]",cc,cc
2,309137749,10,94025.19,CC,NaN,NaN,NaN,NaN,NaN,2018-01-20 19:50:42,...,NaN,cc,vacio,vacio,1,1,1,cc,vacio,vacio
3,324614737,10,94430.07,CC,NaN,NaN,NaN,NaN,NaN,2018-03-26 19:21:46,...,NaN,cc,vacio,vacio,1,1,1,cc,vacio,vacio
4,235344690,18,670645.57,MEDICINA PREPAGADA COLSANITAS,CE,NaN,NaN,NaN,NaN,2017-01-06 20:13:17,...,ce,medicina prepagada colsanitas,ce,ce,3,1,1,"[medicina, prepagada, colsanitas]",ce,ce


Se guardan para futuras consultas

In [51]:
for row in transform_dict_nlp:
    print(row)

referencia contrato valor
medicina prepagada colsanitas
recaudo colsanitas
pago de la planilla cesantias
pago de la factura conjunto residencial pietra santa ph
pago de saldo plan familiar
pago por irecaudo de claro soluciones moviles
pago factura asociado
presentacion y pago del impuesto predial unificado
edif mirabell ph bog
liquidacion iuva no
pago de saldo
referencia de pago express no
referencia pago express no
ref pago express no
pago de retenciones de industria y comercio
servicios virtuales de camara de comercio de bogota
pago electronico emcaliach
pago factura
compra a valorizacion medellin fonvalmed
pago de factura
factura etb cuenta periodo
maquina de terapia digital con electrodos
pago impuesto predial unificado
vdmjx changebooking psepayment pago de tiquete flight fc date
transaccion para generacion de certificados de tradicion y libertad
pago de factura gas natural cupon no
empresas publicas de medellin esp
recaudo colegio
pago de facturas
pago contrato no factura
recaudo

pago a traves del portal transaccional alkosto
pago price travel reserva attxum
edificio torre marfil
pago de los documentos
pago de los documentos vse
cc duplicado
compra de certificados certificado web
derechos de matricula ingenieria de sistemas periodo
pagos desde miavantel a pago de facturas
pago de factura citibank mastercard numero
pago de factura de aaa
pago de factura de energia
factura acueducto alcantarillado
pago facturas m
pago facturas t
pago de recibo preara
pago de cartera
compra de pasaje en pinbuscom production
servicios fondo especial
recarga tarjeta
peter macala
eticket avianca ncuvbw
eticket avianca parsn
eticket avianca wwlb
eticket avianca lpn
limpieza facial con microdermoabrasion con off
paymentid ddbdabadcceeebe
factura del servicio de acueducto alcantarillado facturas
pago eps
pago facturas por internet
compra a universidad pontificia bolivariana
referencia de pago no
leasing chevrolet
eticket avianca kiyc
eticket avianca tym
pago banco de bogota tarjeta de c

live trading camp estudiantes
live trading camp ltc super descuento
concepto de pago
pago mapfre seguros colombia
eticket avianca jxvms
pago price travel reserva wfnfhr
contractid addreess cl cr
compra de productos deli reposteria
redbus payment
obmqk booking psepayment pago de tiquete flight fc date ob
pago rentabien sas
contractid addreess cr cl b sur interior
eticket avianca qmiwdb
expreso brasilia sa compra de tiquetes
pago de factura mitigo app
contractid addreess cl a cr b interior
pago price travel reserva ilbpct
canon cl ap b edif park avenue
eticket avianca pkjxa
contractid addreess cr b cl f sur inter
administracion ene
paymentid bfbdaccf
eticket avianca lknelv
pago de matricula
eticket avianca vvdmbm
inscripcion unicef k categoria abierta documento
mueble espejo zapatero de niveles color blanco
contractid addreess blq apto b
cbvmh changebooking psepayment pago de tiquete flight fc date
recaudo pse v voluntario cooperante
compra certificados
renovacion virtual
impuesto sobre 

factura liceo
cedula de ciudadania
pago tarjeta retail davivienda
bog bog
bog bog bog
contractid addreess cl a sur cr interi
nrnvv booking psepayment pago de tiquete flight fc date n
para la compra de minutos
diumtb bogrw
giro realizado desde el app
contractid addreess cl b cr b interior
contractid addreess cl a cr interior
entradas de mil y servicio
lg q pse
compra a atletico nacional abonados
inscripciones pregrado
cristian arboleda
colegio gimnasio pinares diciembre
contractid addreess cl a
contractid addreess cr cl interior
tai chi chicung avanzado jad
tburwa booking psepayment pago de tiquete flight fc date tb
contractid addreess cl b sur cr e interior
contractid addreess cl unico liborina
deposito al martillo con cc y nombre jesus galeano
pago de liquidacion licenciatura en lengua castellana
contractid addreess cr
pagos a yanbal realizados por pse
pago banco av villas tarjeta de credito mastercard
cuota mes de mayo
soat colpatria
contractid addreess cl cr ac interior
olivares ene

uno internacional sebastian bustamante vargas colegio militar decroly n
compra de ticket s
valor emsa
pago de recibo preins
abono a cuenta de ahorros
ba corporativo mbp
servicio pago de matricula de pregrado
miguel ricardo garcia rivero
a ingl s principiante n jad
pago impuesto de vehiculos
pago referencia
contractid addreess mza csa sm
recarga para el celular del usuario leydertacuma gmailcom
compra en aquarella ventas por catalogo
pago de servicio
eticket avianca knqb
servicio mes octubre
pagodonacion con pse
pago pensi n y almuerzo mes febrero
pago conjunto residencial origami ph
pagos de la tienda lte
contractid addreess kr apto mirador d
eticket avianca ymwri
udsvx booking psepayment pago de tiquete flight fc date ud
para acreditar dinero a la cuenta
eticket colaborador avianca jryur
canon de arrendamiento
contractid addreess cl kr
consumo regular
impuesto de vehiculos departamento de caldasdood
paymentid fcbeaaeeeae
mic ponemos la fantasia
cancela febrero
recaudo vales electronic

...

dbhx booking psepayment pago de tiquete flight fc date db
tdvk booking psepayment pago de tiquete flight fc date td
validacion de asignaturas
eticket avianca jkjkdg
iugg booking psepayment pago de tiquete flight fc date i
pago price travel reserva jrrkmt
jzpz booking psepayment pago de tiquete flight fc date j
los esperamos este de abril devuelta a nuestra tarima el
set de relojes en modelo a eleccion
ktronix compra en linea celular libre moto c plus g dorado
four brothers
pension a octubre
compra tickete
eticket avianca lhm
juego de paintball para o personas o bolas
paymentid ffababcdcdd
reserva mioraco en alvaro pedraza peluqueria sede restrepo
nlujd booking psepayment pago de tiquete flight fc date n
zkt booking psepayment pago de tiquete flight fc date z
paymentid abbfcbabeba
ktronix compra en linea consola ps hits v slim tera control jueg
fragancia nautica para hombre a eleccionczlbl changebooking psepayment pago de tiquete flight fc date
alfredo torres chaux
faja cachetera strapl

diciembre medellin
paymentid fbaaedbdfcb
zf booking psepayment pago de tiquete flight fc date z
fecmb booking psepayment pago de tiquete flight fc date fe
atbtg changebooking psepayment pago de tiquete flight fc date
czwsl booking psepayment pago de tiquete flight fc date cz
orden customer carolpulgarin hotmailcom total
compra a inmobiliaria royal micrositio recaudo abierto coplina
qgls changebooking psepayment pago de tiquete flight fc date
hccfz booking psepayment pago de tiquete flight fc date h
vuelos bog a mde kgmmj
eticket avianca jregkw
tenis spa lona rayas rosadas
fraventa servicios
xdvdsx changebooking psepayment pago de tiquete flight fc date
eticket avianca pwtpkc
soat placardde
dks booking psepayment pago de tiquete flight fc date d
funda cargador para iphone en referencia y color a eleccion
clara rosa cortes gutierrez
eticket avianca vhigx
paymentid eceebeeab
horas en suite especial con off
bzvzl booking psepayment pago de tiquete flight fc date bz
ourz booking psepayment 

cyqqy booking psepayment pago de tiquete flight fc date c
ccyi booking psepayment pago de tiquete flight fc date c
paymentid dbcbbaebead
contractid addreess cr c cl ac interior
tarmac lanza mash dem down sabado de noviembre de gener
cuota inicial madera fresca
solicitud historico vehicular para el vehiculo de placa trh
ejgl changebooking psepayment pago de tiquete flight fc date
paymentid ebcacafbfbbffac
cd booking psepayment pago de tiquete flight fc date c
pago de reserva de pista de bolos
solicitud historico vehicular para el vehiculo de placa mgz
paymentid cdeaafdcfcb
contractid addreess avda a diag aa int
ruta ampm
eticket avianca vohdn
eticket avianca pteg
ykkg booking psepayment pago de tiquete flight fc date y
paymentid abaffddaaeecbae
cofre african safari extracontenido sofi burbujas cofre elige se
pago marzo nhc
cuota nhc abril
solicitud historico vehicular para el vehiculo de placa goc
canon tv a ap septiembre de
gloria patricia mazo gomez
paymentid fdedabfabffcf
paymentid d

orden customer mauroramirez hotmailcom total
order no d in http undulcedespertarco
pago price travel reserva geywdk
respirador mascarilla de libre mantenimiento n m
contractid addreess dg bl ap
o burritos a eleccion bebida con off
eticket avianca siwji
inscripci n conferencia icpr am ricas
eticket avianca ztqcm
paymentid bcdbdecabaf
revnr booking psepayment pago de tiquete flight fc date re
fzrel booking psepayment pago de tiquete flight fc date fz
solera et
tickets burgers beers bikes night ride ed
eticket avianca onehcy
eticket avianca oaji
taller de agricultura organica mi primera siembrapropagacion sabado de
ktronix compra en linea pila recargable sony mah aa x
picada mexicana o picada bbq limonada para dos o cuatro con off
eticket avianca nyisn
solicitud historico vehicular para el vehiculo de placa gxp
modulo nocturno chapinero
pago price travel reserva inbepj
recarga para el celular del usuario carolinaguilera gmail
permiso de carga pesada y extradimensionada a placa skl
lqpc bo

KeyboardInterrupt: 

In [62]:
stored_dict_ref1 = {}
for keys in transform_dict_nlp.keys():
    stored_dict_ref1[keys] = [transform_dict_nlp[keys]]

In [63]:
stored_dict_ref2 = {}
for keys in transform_dict_nlp_ref2.keys():
    stored_dict_ref2[keys] = [transform_dict_nlp_ref2[keys]]

In [64]:
stored_dict_ref3 = {}
for keys in transform_dict_nlp_ref3.keys():
    stored_dict_ref3[keys] = [transform_dict_nlp_ref3[keys]]

In [65]:
ref1_dict = pd.DataFrame.from_dict(stored_dict_ref1,orient='index')
ref2_dict = pd.DataFrame.from_dict(stored_dict_ref2,orient='index')
ref3_dict = pd.DataFrame.from_dict(stored_dict_ref3,orient='index')

In [67]:
data.dtypes

id_trn_ach                               object
id_cliente                               object
valor_trx                               float64
ref1                                     object
ref2                                     object
ref3                                    float64
sector                                   object
subsector                                object
descripcion                              object
fecha_usable                     datetime64[ns]
ref1_notilde                             object
ref2_notilde                             object
ref3_notilde                             object
ref1_notilde_noespecial                  object
ref2_notilde_noespecial                  object
ref3_notilde_noespecial                  object
ref1_notilde_noespecial_vacio            object
ref2_notilde_noespecial_vacio            object
ref3_notilde_noespecial_vacio            object
largo_ref1                                int64
largo_ref2                              

In [68]:
data.columns

Index(['id_trn_ach', 'id_cliente', 'valor_trx', 'ref1', 'ref2', 'ref3',
       'sector', 'subsector', 'descripcion', 'fecha_usable', 'ref1_notilde',
       'ref2_notilde', 'ref3_notilde', 'ref1_notilde_noespecial',
       'ref2_notilde_noespecial', 'ref3_notilde_noespecial',
       'ref1_notilde_noespecial_vacio', 'ref2_notilde_noespecial_vacio',
       'ref3_notilde_noespecial_vacio', 'largo_ref1', 'largo_ref2',
       'largo_ref3', 'ref1_exp_nlp', 'ref2_exp_nlp', 'ref3_exp_nlp'],
      dtype='object')

# Es evidente que necesita una depuracion de columnas

In [70]:
data_to_write = data.drop(['ref1_notilde',
       'ref2_notilde', 'ref3_notilde', 'ref1_notilde_noespecial',
       'ref2_notilde_noespecial', 'ref3_notilde_noespecial',
       'ref1_notilde_noespecial_vacio', 'ref2_notilde_noespecial_vacio',
       'ref3_notilde_noespecial_vacio', 'largo_ref1', 'largo_ref2',
       'largo_ref3'],axis=1)

In [71]:
data_to_write.dtypes

id_trn_ach              object
id_cliente              object
valor_trx              float64
ref1                    object
ref2                    object
ref3                   float64
sector                  object
subsector               object
descripcion             object
fecha_usable    datetime64[ns]
ref1_exp_nlp            object
ref2_exp_nlp            object
ref3_exp_nlp            object
dtype: object

In [72]:
ref1_dict.to_pickle('trxpse_ref1_dict.pkl.gzip')
ref2_dict.to_pickle('trxpse_ref2_dict.pkl.gzip')
ref3_dict.to_pickle('trxpse_ref3_dict.pkl.gzip')
data_to_write.to_pickle('dt_trxpse_V3.pkl.gzip')

# Con esto se termina el V3. Para la proxima vamos a hacer un enhancement de la data por medio de features de clientes.